In [2]:
import numpy as np 
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy.linalg import det, svd, inv, norm
from scipy import stats
import random
from sklearn.naive_bayes import GaussianNB

## PCA + KNN Classifier (poses)
def knn_poses(split_ratio, k_value):
    PCA_DIM=10
    CLASSES=5
    SPLIT_RATIO=split_ratio
    K_VALUE=k_value

    pose_data = loadmat('pose.mat')
    pose_data['pose'].shape

    poses = np.zeros((884,1920))
    for i in range(0,884):
        poses[i] = pose_data['pose'][:,:,i%13,i//13].reshape([1920])


    U, sigma, V = svd(poses.transpose())
    Y = np.dot(poses, U[:,0:PCA_DIM])


    TRAIN=round(13*SPLIT_RATIO)
    TEST=13-TRAIN

    TRAIN_SAMPLES=CLASSES*TRAIN
    TEST_SAMPLES=CLASSES*TEST

    train_poses = np.zeros((TRAIN_SAMPLES, PCA_DIM))
    test_poses = np.zeros((TEST_SAMPLES, PCA_DIM))

    test_labels = np.zeros((TEST_SAMPLES))
    train_labels = np.zeros((TRAIN_SAMPLES))


    for i in range(0,CLASSES):
        train_poses[TRAIN*i:TRAIN*i+TRAIN] = Y[13*i:13*i+TRAIN]
        test_poses[TEST*i:TEST*i+TEST] =  Y[13*i+TRAIN:13*i+13]
        test_labels[TEST*i:TEST*i+TEST] = np.ones((TEST)) *  i
        train_labels[TRAIN*i:TRAIN*i+TRAIN] =np.ones((TRAIN)) *  i

    def find_neighbours(dataset, new_data, k):
        distances = []
        for idx, data in enumerate(dataset):
            distances.append(norm(data - new_data))
        return np.array(sorted(range(len(distances)), key=lambda i: distances[i])[:k])

    def predict_labels(train_labels, neighbours):
        p_label = np.zeros((neighbours.size))
        for i, n in enumerate(neighbours):
            p_label[i] = train_labels[n]
        return stats.mode(p_label).mode[0]

    dataset = train_poses

    right=0
    wrong=0
    for idx, test in enumerate(test_poses):
        neighbours = find_neighbours(dataset, test, K_VALUE)
        mode = predict_labels(train_labels, neighbours)
        if mode == test_labels[idx]:
            right+=1
        else:
            wrong+=1

    print("Split Ratio: {} K-value: {}".format(SPLIT_RATIO, K_VALUE))
    print("Correct : {} Incorrect : {} Accuracy : {} %".format(right, wrong, right/TEST_SAMPLES * 100))
    
for split_ratio in [0.95, 0.90, 0.80, 0.75]:
    for k_value in [1,2,3,4,5,10,15]:
        knn_poses(split_ratio,k_value)

Split Ratio: 0.95 K-value: 1
Correct : 4 Incorrect : 1 Accuracy : 80.0 %
Split Ratio: 0.95 K-value: 2
Correct : 3 Incorrect : 2 Accuracy : 60.0 %
Split Ratio: 0.95 K-value: 3
Correct : 2 Incorrect : 3 Accuracy : 40.0 %
Split Ratio: 0.95 K-value: 4
Correct : 2 Incorrect : 3 Accuracy : 40.0 %
Split Ratio: 0.95 K-value: 5
Correct : 2 Incorrect : 3 Accuracy : 40.0 %
Split Ratio: 0.95 K-value: 10
Correct : 3 Incorrect : 2 Accuracy : 60.0 %
Split Ratio: 0.95 K-value: 15
Correct : 2 Incorrect : 3 Accuracy : 40.0 %
Split Ratio: 0.9 K-value: 1
Correct : 4 Incorrect : 1 Accuracy : 80.0 %
Split Ratio: 0.9 K-value: 2
Correct : 3 Incorrect : 2 Accuracy : 60.0 %
Split Ratio: 0.9 K-value: 3
Correct : 2 Incorrect : 3 Accuracy : 40.0 %
Split Ratio: 0.9 K-value: 4
Correct : 2 Incorrect : 3 Accuracy : 40.0 %
Split Ratio: 0.9 K-value: 5
Correct : 2 Incorrect : 3 Accuracy : 40.0 %
Split Ratio: 0.9 K-value: 10
Correct : 3 Incorrect : 2 Accuracy : 60.0 %
Split Ratio: 0.9 K-value: 15
Correct : 2 Incorrect : 3